In [62]:
import pandas as pd
import requests
import json
import matplotlib as mlt
import gmaps
from census import Census
from us import states


census_API = 'ad9e020616792d8cbe0f666089dc3980b9d379e0'

#KCMO 311 Dataset
path = './Resources/311_Pothole_History.csv'

df = pd.DataFrame(pd.read_csv(path))
df.head(2)


,CASE ID,SOURCE,DEPARTMENT,WORK GROUP,REQUEST TYPE,CREATION MONTH,CREATION YEAR,STATUS,EXCEEDED EST TIMEFRAME,CLOSED DATE,...,CLOSED YEAR,DAYS TO CLOSE,STREET ADDRESS,ADDRESS WITH GEOCODE,ZIP CODE,NEIGHBORHOOD,COUNCIL DISTRICT,PARCEL ID NO,LATITUDE,LONGITUDE
0,2019169222,PHONE,Public Works,Public Works-Street and Traffic-District 3,Streets / Roadways / Alleys-Pothole-District 3,9,2019,RESOL,Y,11/7/2019,...,2019.0,48.0,13110 E 57th St,"13110 E 57th St64133\n(39.018605, -94.426046)",64133.0,Blue Vue Hills,5.0,63787,39.018605,-94.426046
1,2019126105,PHONE,Public Works,Public Works-Street and Traffic-District 3,Streets / Roadways / Alleys-Pothole-District 3,7,2019,RESOL,Y,10/24/2019,...,2019.0,113.0,1420 E 75th Ter,"1420 E 75th Ter64131\n(38.990601, -94.570141)",64131.0,East Meyer 6,5.0,115894,38.990601,-94.570141


In [71]:
#Zip code list
zip_codes = df['ZIP CODE'].dropna().unique()
zip_codes = zip_codes.astype(int)

In [79]:

census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E","B19301_001E","B17001_002E"), {'for': 'zip code tabulation area:'+str(66203)})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * census_pd["Poverty Count"].astype(int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate"]]


# Visualize
print(len(census_pd))
census_pd.head()

1


,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,66203,19290.0,39.8,56250.0,30571.0,1970.0,10.212545
